In [27]:
# XML, TEI, CMI/F and data handling
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/) for XML
from bs4 import Comment # BS4-addon for å håndtere kommentarer <!-- X -->
import re # Regex
import pandas as pd
import collections # Facilitate dynamic dict

# Time and date
import datetime # Dates
from datetime import date
import time # Time

# File and folder handling
import glob # The yeast of thought and mind
import os # Filsystem; mapper, lagring, åpning, etc...
import shutil # Se os+

from string import punctuation

In [8]:
version = "X2" # Describes the "program's" state of completion and versioning.
today = date.today() # Sett dato i dag
today = today.strftime("%Y-%m-%d") # Formater dato
currVer = version

previouslyRun = "Last executed code was version "+str(currVer)+". All OUTPUT files are current to that version on that date.\n"+str(cmifUid)+"."
print("Version",currVer)
cmifUid = "a403c593-09df-4538-8acf" # Unique ID. Used in sourceDesc of CMIF. Don't change it without a good reason.
# cmifUid is also used as "source" for the time being in each object. Read more about this in CMIF docs.

Version X2


# *MunchXMLmuncher* **X2**
MXMLMx2 reads and preprocesses files.

In [62]:
hasCorrespondenceXML,hasTEIXML,hasChrono,hasXMLs = False,False,False,False
programfolder = "MXMLM "+currVer
if os.path.exists(programfolder):
    print("Found directory '% s'" % programfolder)
else:
    os.mkdir(programfolder)
    print("Directory '% s' created" % programfolder)
inputfolder = os.path.join(programfolder,"sourcefiles") # The folder containing the TEI/XML-files to be transformed.
paths = ['correspondence.xml','register_tei.xml']
if not os.path.exists(inputfolder):
    os.mkdir(inputfolder)
    print("Directory '% s' created" % inputfolder)
outputfolder = os.path.join(programfolder,"output") # Output folder
if not os.path.exists(outputfolder):
    os.mkdir(outputfolder)
    print("Directory '% s' created" % outputfolder)



# Chrono
lookupChrono = sorted(glob.glob("*Kronologi_Munchs_brev*.xlsx"), key=os.path.getmtime)
x = len(lookupChrono)-1
if x > -1:
    print("Newest chronology file:",lookupChrono[x])
    chronologyFile = lookupChrono[x]
    hasChrono = True
    shutil.copy2(chronologyFile, inputfolder+"/kronologi.xlsx")
    chronologyFile = inputfolder+"/kronologi.xlsx"
    chronology = pd.read_excel(chronologyFile).dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    chronology = chronology.fillna("N/A")
    
    CHRONODICT = collections.defaultdict(dict)
    for idx,row in chronology.iterrows():
        document = chronology.iloc[idx]['Objektnr.']
        rawdate = chronology.iloc[idx]['Dato']
        if rawdate != "N/A" and document != "N/A": # If date and documents are not N/A
            chkStr = document[0:2]
            if chkStr == "PN":
                #print(document)
                document = document.replace(" ","0")
                if len(document) == 5:
                    filenameEdited=document[:2] + "0" + document[len(document)-3:]
                else:
                    filenameEdited = document[:2] +"000"+ document[len(document)-4:]
                    filenameEdited = filenameEdited[:2] + filenameEdited[len(filenameEdited)-4:]
                filenamePlain = filenameEdited[0:6]
            elif chkStr == "MM":
                #No-MM_K0499
                #filenamePlain = document[0:11]
                if len(document) == 8:
                    #print(document)
                    filenamePlain = document[:2]+"_"+document[3]+"0"+document[5:]
                else:
                    filenamePlain = document[:2]+"_"+document[3]+document[5:]
                filenamePlain = "No-"+filenamePlain
            elif chkStr == "M ": # Implemented to catch "M N 2714"...
                filenamePlain = document[:1]+"_"+document[2]+document[4:]
                filenamePlain = "No-M"+filenamePlain
                #filenamePlain = filenamePlain.replace(" ","")
                print("Caught",document,">",filenamePlain)
            elif chkStr == "N ": # Implemented to catch "N 3491"...
                  # This is ASSUMED to be "No-MM_N3491" and not PN3491
                filenamePlain = "No-MM-N_"+document[2:]
                print("Caught",document,">",filenamePlain)
            elif chkStr == " M": # Implemented to catch " MM N 723"...
                filenamePlain = document[1:3]+"_"+document[4]+"0"+document[6:]
                print("Caught",document,">",filenamePlain)
            else:
                print("!!\tError in",document,"length:",len(document))
                filenamePlain = "Formatting error"
            #print(filenamePlain)
            
            document = filenamePlain
            if isinstance(rawdate,datetime.date): # If it's just a datetime object
                dateobject = rawdate.strftime("%Y-%m-%d")
            else:
                #print(document,rawdate)
                dateobject = str(rawdate) # Make sure it's string
                
                string4print = document+" "+dateobject
                
                dateobject = dateobject.replace("–","-") # Replace long dash – with a normal dash -
                if ".-" in dateobject: # Like: 04.-05.1922.
                    dateobject = re.sub(r'^.*?\.-', '', dateobject, flags=re.DOTALL) # Remove EVERYTHING to the left of it.
# If you want date ranges, use complete dates separated by "-". Do not give me ".-".
                if "?" in dateobject:
                    dateobject = dateobject.replace("?","").strip(punctuation) # Remove n ?s and then also remove excess .

                    dateobject = dateobject.replace("..",".")
                if "-" in dateobject:
                    splitToFrom = dateobject.split("-")
                    fromDate = splitToFrom[0]
                    toDate = splitToFrom[1]
                    newFromDate = fromDate.split(".")
                    if isinstance(newFromDate,list):
                        newdate = newFromDate[len(newFromDate)-1]
                        for x in reversed(newFromDate):
                            if len(x) == 4:
                                pass
                            else:
                                if len(x) == 2:
                                    newdate+="-"+x
                                else:
                                    break  
                    else:
                        newdate = fromDate
                    newdate+="%"
                    newToDate = toDate.split(".")
                    if isinstance(newToDate,list):
                        newdateA = newToDate[len(newToDate)-1]
                        for x in reversed(newToDate):
                            if len(x) == 4:
                                pass
                            else:
                                if len(x) == 2:
                                    newdateA+="-"+x
                                else:
                                    break  
                    else:
                        newdateA = toDate
                    newdate+=newdateA
                    #print("yolo",document)
                else:
                    datelements = dateobject.split(".")
                    newdate = datelements[len(datelements)-1]
                    for x in reversed(datelements):
                        if len(x) == 4:
                            pass
                        else:
                            if len(x) == 2:
                                newdate+="-"+x
                            else:
                                break      
                print(document,rawdate,"("+newdate+")")
            CHRONODICT[document]["date"] = newdate
        #if type(rawdate) == datetime.datetime:
        #print(document,rawdate,type(rawdate))
#print(CHRONODICT)

Found directory 'MXMLM X2'
Newest chronology file: Kronologi_Munchs_brev_20220923.xlsx
No-MM_N01148 ??.11.1932 (1932-11)
No-MM_N00275 ??.11.1932 (1932-11)
No-MM_N01697 ??.10.1932 (1932-10)
No-MM_N02539 ??.08.1932 (1932-08)
PN1428 ??.06.1932 (1932-06)
PN1421 ??.06.1932 (1932-06)
No-MM_N01149 ??.04.1932 (1932-04)
PN1429 2?.02.1932 (1932-02)
No-MM_N03376 2?.02.1932 (1932-02)
PN0690 ??.01..1932 (1932-01)
No-MM_N03121 ??.01..1932 (1932-01)
<class 'list'> ['1931', '1933']
No-MM_N01728 1931-1933 (1931%1933)
<class 'list'> ['1931', '1933']
No-MM_N01729 1931-1933 (1931%1933)
No-MM_N01668 ??.??.1931 (1931)
PN1437 ??.12.1931 (1931-12)
No-MM_N02778 ??.12.1931 (1931-12)
No-MM_N02538 ??.12.1931 (1931-12)
No-MM_N02022 10.-11.1931 (1931-11)
PN0688 ??.10.1931 (1931-10)
PN1096 ??.10.1931 (1931-10)
PN1094 ??.06.1931 (1931-06)
No-MM_N03752 ??.06.1931 (1931-06)
No-MM_N01145 ??.05.1931 (1931-05)
No-MM_N01143 ??.05.1931 (1931-05)
PN0097 ??.02.1931 (1931-02)
PN0092 01.-02.1931 (1931-02)
No-MM_N02535 01.-02.19

No-MM_N3273 ??.04.1915 (1915-04)
No-MM_N1065 ??.03.1915 (1915-03)
No-MM_N1091 ??.03.1915 (1915-03)
No-MM_N3243 ??.02.1915 (1915-02)
No-MM_N1015 ??.02.1915 (1915-02)
PN0600 ??.01.1915 (1915-01)
PN0605 ??.01.1915 (1915-01)
<class 'list'> ['1914', '1915']
No-MM_N1089 1914-1915 (1914%1915)
PN0599 ??.12.1914 (1914-12)
No-MM_N3240 ??.12.1914 (1914-12)
No-MM_N1041 ??.12.1914 (1914-12)
No-MM_N2760 ??.??.1914 (1914)
No-MM_N2027 ??.09.1914 (1914-09)
No-MM_N1797 ??.09.1914 (1914-09)
No-MM_N0710 ??.09.1914 (1914-09)
PN0597 ??.09.1914 (1914-09)
No-MM_N3238 ??.06.1914 (1914-06)
No-MM_N1406 ??.??.1914 (1914)
No-MM_N2455 03.-04.1914 (1914-04)
PN0251 ??.03.1914 (1914-03)
No-MM_N2279 ??.03.1914 (1914-03)
No-MM_N3515 ??.03.1914 (1914-03)
PN0593 ??.02.1914 (1914-02)
No-MM_N3069 ??.12.1913 (1913-12)
PN0763 ??.12.1913 (1913-12)
PN0592 ??.12.1913 (1913-12)
PN0591 ??.12.1913 (1913-12)
PN1211 ??.11.1913 (1913-11)
No-MM_N3429 ??.11.1913 (1913-11)
No-MM_N2456 ??.11.1913 (1913-11)
No-MM_N2454 ??.11.1913 (1913-11)

No-MM_N3333 07.-08.1906 (1906-08)
PN0077 07.-08.1906 (1906-08)
No-MM_N0908 ??.07.1906 (1906-07)
PN0407 ??.07.1906 (1906-07)
No-MM_N2451 ??.07.1906 (1906-07)
No-MM_N2295 ??.07.1906 (1906-07)
No-MM_N2294 ??.??.1906 (1906)
PN0075 ??.07.1906 (1906-07)
No-MM_N1802 ??.07.1906 (1906-07)
No-MM_N1803 ??.07.1906 (1906-07)
No-MM_N0789 ??.06.1906 (1906-06)
No-MM_N0917 ??.06.1906 (1906-06)
PN0405 ??.06.1906 (1906-06)
PN0071 ??.??.1906 (1906)
PN1028 ??.06.1906 (1906-06)
No-MM_N0913 ??.06.1906 (1906-06)
PN0076 ??.06.1906 (1906-06)
PN0403 05.-06.1906 (1906-06)
PN0402 ??.05.1906 (1906-05)
PN0074 ??.05.1906 (1906-05)
PN0401 ??.05.1906 (1906-05)
No-MM_N2567 ??.??.1906 (1906)
PN1331 ??.??.1906 (1906)
Caught M N 2714 > No-MM_N2714
No-MM_N2714 ??.??.1906 (1906)
PN0400 ??.05.1906 (1906-05)
No-MM_N2563 ??.05.1906 (1906-05)
No-MM_N3471 ??.05.1906 (1906-05)
No-MM_N2557 ??.05.1906 (1906-05)
No-MM_N3360 ??.05.1906 (1906-05)
No-MM_N2187 ??.05.1906 (1906-05)
No-MM_N2188 ??.05.1906 (1906-05)
PN0153 ??.04.1906 (1906-

<class 'list'> ['1896', '1897']
PN0045 1896-1897 (1896%1897)
No-MM_N0553 12.1896 (1896-12)
<class 'list'> ['1896', '1897']
No-MM_N3400 1896-1897 (1896%1897)
No-MM_N2757 20.12.1896 (1896-12-20)
No-MM_N0824 11.1896 (1896-11)
PN1003 29.10.1896 (1896-10-29)
No-MM_N0822 1896 (1896)
No-MM_N0823 16.10.1896 (1896-10-16)
PN1059 30.09.1896 (1896-09-30)
No-MM_K4532 14.09.1896 (1896-09-14)
No-MM_N0820 09.1896 (1896-09)
No-MM_N0970 13.08.1896 (1896-08-13)
No-MM_N0821 04.08.1896 (1896-08-04)
No-MM_N2042 1896 (1896)
No-MM_N2044 1896 (1896)
No-MM_N1722 1896 (1896)
No-MM_N1720 08.1896 (1896-08)
No-MM_N2040 1896 (1896)
No-MM_N1721 1896 (1896)
No-MM_N2282 1896 (1896)
No-MM_N0188 08.1896 (1896-08)
PN0268 07.08.1896 (1896-08-07)
PN0267 25.07.1896 (1896-07-25)
PN1376 ??.07.1896 (1896-07)
No-MM_N0819 01.07.1896 (1896-07-01)
PN0266 29.06.1896 (1896-06-29)
No-MM_K0311 27.06.1896 (1896-06-27)
PN0209 06.1896 (1896-06)
No-MM_N2420 06.1896 (1896-06)
No-MM_K4529 23.06.1896 (1896-06-23)
No-MM_N1100 06.1896 (1896-06)

In [63]:
CHRONODICT

defaultdict(dict,
            {'No-MM_N03101': {'date': '1874-06-18'},
             'PN0888': {'date': '1874-06-18'},
             'No-MM_N02499': {'date': '1874-06-18'},
             'No-MM_N01151': {'date': '1874-06-18'},
             'No-MM_N01148': {'date': '1932-11'},
             'PN1420': {'date': '1932-11'},
             'No-MM_N02548': {'date': '1932-11'},
             'No-MM_N00275': {'date': '1932-11'},
             'No-MM_N01683': {'date': '1932-11'},
             'No-MM_N01685': {'date': '1932-11'},
             'No-MM_N01697': {'date': '1932-10'},
             'PN1426': {'date': '1932-10'},
             'No-MM_N03377': {'date': '1932-10'},
             'PN1117': {'date': '1932-10'},
             'PN0134': {'date': '1932-10'},
             'PN1446': {'date': '1932-10'},
             'PN0006': {'date': '1932-10'},
             'No-MM_N02539': {'date': '1932-08'},
             'PN1419': {'date': '1932-08'},
             'PN0133': {'date': '1932-08'},
             'PN0132': {

In [9]:
hasCorrespondenceXML,hasTEIXML,hasChrono,hasXMLs = False,False,False,False
programfolder = "MXMLM "+currVer
if os.path.exists(programfolder):
    print("Found directory '% s'" % programfolder)
else:
    os.mkdir(programfolder)
    print("Directory '% s' created" % programfolder)
inputfolder = os.path.join(programfolder,"sourcefiles") # The folder containing the TEI/XML-files to be transformed.
paths = ['correspondence.xml','register_tei.xml']
if not os.path.exists(inputfolder):
    os.mkdir(inputfolder)
    print("Directory '% s' created" % inputfolder)
outputfolder = os.path.join(programfolder,"output") # Output folder
if not os.path.exists(outputfolder):
    os.mkdir(outputfolder)
    print("Directory '% s' created" % outputfolder)



# Chrono
lookupChrono = sorted(glob.glob("*Kronologi_Munchs_brev*.xlsx"), key=os.path.getmtime)
x = len(lookupChrono)-1
if x > -1:
    print("Newest chronology file:",lookupChrono[x])
    chronologyFile = lookupChrono[x]
    hasChrono = True
    shutil.copy2(chronologyFile, inputfolder+"/kronologi.xlsx")
    chronologyFile = inputfolder+"/kronologi.xlsx"
    chronology = pd.read_excel(chronologyFile).dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    chronology = chronology.fillna("N/A")
    
    CHRONODICT = collections.defaultdict(dict)
    for idx,row in chronology.iterrows():
        document = chronology.iloc[idx]['Objektnr.']
        rawdate = chronology.iloc[idx]['Dato']
    
    # This step requires several customised workarounds to handle the irregular and non-machine-readable data present in
    # the date file. As a result, it can accept a broad range of dates.

    j = []
    datesNotAdded = []
    dupes = []
    chronology = pd.read_excel(chronologyFile).dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    chronology = chronology.fillna("N/A")
    try:
        if placenamedict:
            print("Placenamedict found with content")
        else:
            print("Placenamedict found without content")
    except:
        placenamedict = collections.defaultdict(dict)
        print("Made new dict")
    i=0
    for idx,row in chronology.iterrows():
        document = chronology.iloc[idx]['Objektnr.']
        skipThis = False
        if document != "N/A":
            chkStr = document[0:2]
            if chkStr == "PN":
                #print(document)
                document = document.replace(" ","0")
                if len(document) == 5:
                    filenameEdited=document[:2] + "0" + document[len(document)-3:]
                else:
                    filenameEdited = document[:2] +"000"+ document[len(document)-4:]
                    filenameEdited = filenameEdited[:2] + filenameEdited[len(filenameEdited)-4:]
                filenamePlain = filenameEdited[0:6]
            elif chkStr == "MM":
                #No-MM_K0499
                #filenamePlain = document[0:11]
                if len(document) == 8:
                    #print(document)
                    filenamePlain = document[:2]+"_"+document[3]+"0"+document[5:]
                else:
                    filenamePlain = document[:2]+"_"+document[3]+document[5:]
                filenamePlain = "No-"+filenamePlain
            elif chkStr == "M ": # Implemented to catch "M N 2714"...
                filenamePlain = document[:1]+"_"+document[2]+document[4:]
                filenamePlain = "No-M"+filenamePlain
                #filenamePlain = filenamePlain.replace(" ","")
                print("Caught",document,">",filenamePlain)
            elif chkStr == "N ": # Implemented to catch "N 3491"...
                  # This is ASSUMED to be "No-MM_N3491" and not PN3491
                filenamePlain = "No-MM-N_"+document[2:]
                print("Caught",document,">",filenamePlain)
            elif chkStr == " M": # Implemented to catch " MM N 723"...
                filenamePlain = document[1:3]+"_"+document[4]+"0"+document[6:]
                print("Caught",document,">",filenamePlain)
            else:
                print("!!\tError in",document,"length:",len(document))
                filenamePlain = "Formatting error"
            #print(filenamePlain)

            if filenamePlain != "Formatting error":
                #date = chronology.iloc[i]['Dato']
                if filenamePlain in j:
                    dupes.append(filenamePlain)
                    #print("Found duplicate",filenamePlain)
                else:
                    j.append(filenamePlain)
                ### This indent for dates after fixing the naming conventions
                fromMonth,fromYear,toMonth,toYear,dateobject,Year,fromDate,toDate = False,False,False,False,False,False,False,False
                #newYear,newMonth,newDay = "","",""
                dateobject = chronology.iloc[idx]['Dato']

                if dateobject != "N/A": # If date object isn't N/A
                    if isinstance(dateobject,datetime.date): # If it's just a datetime object
                        dateobject = dateobject.strftime("%Y-%m-%d") # We're done! Yay!
                        #print(idx,"datetime DATE:",dateobject)
                    else: # If it's not a datetime object:
                        dateobject = str(dateobject)
                        #if ".." in dateobject: 
                        dateobject = dateobject.replace("..",".") # Sometimes there's double dots (..)...
                        dateobject = dateobject.replace("??.","") # Remove trash dates
                        if "?" in dateobject[0:2]:
                            dateobject = dateobject[3:]
                        if "?" in dateobject[0:2]:
                            dateobject = dateobject[3:]
                        #chkDt = dateobject[0:2] # First 2 letters
                        #dateobject = dateobject.replace("/","-")
                        #dateobjectlist = dateobject.split(".")
                        dateobject = dateobject.replace("–","-") # Yes, there are a few dates separated by long dash...
                        if "-" in dateobject: # Used for cases where there may be - or / used as separator.
                            sep = "-"
                        else:
                            sep = "/"
                        if re.match("^([0]?[1-9]|[1|2][0-9]|[3][0|1])[./-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$",dateobject):
                            # Full date
                            dateobjectsplit = dateobject.split(".")
                            dateobject = dateobjectsplit[2]+"-"+dateobjectsplit[1]+"-"+dateobjectsplit[0]
                            if len(dateobjectsplit[2]) != 4: # Only allow 4-digit years.
                                #print(idx,"COMPLETE DATE:",dateobject)
                                skipThis = True
                           # else: # If it does not match
                              #  skipThis = True
                        elif re.match("^([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$",dateobject): # Month.Year
                            dateobjectsplit = dateobject.split(".")
                            dateobject = dateobjectsplit[1]+"-"+dateobjectsplit[0]
                            dateobject = dateobject#+"-01"
                            if len(dateobjectsplit[1]) != 4: # Only allow 4-digit years.
                                #print(idx,"MONTH-YEAR DATE:",dateobject)
                                skipThis = True
                            #else: # If it does not match
                                #skipThis = True 
                        elif re.match("^([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})[./-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$",dateobject): # Month.Year
                            # MM.YYYY/MM.YYYY
                            # MM.YYYY-MM.YYYY
                            dateobjectsplit = dateobject.split(sep)
                            dateobjectsplitFrom = dateobjectsplit[0].split(".")
                            dateobjectsplitTo = dateobjectsplit[1].split(".")
                            fromMonth = dateobjectsplitFrom[0]
                            fromYear = dateobjectsplitFrom[1]
                            toMonth = dateobjectsplitTo[0]
                            toYear = dateobjectsplitTo[1]
                            fromDate = fromYear+"-"+fromMonth
                            toDate = toYear+"-"+toMonth
                            #dateobject = dateobjectsplit[1]+"-"+dateobjectsplit[0]
                            #dateobject = dateobject+"-01"
                            #if len(dateobjectsplit[1]) == 4: # Only allow 4-digit years.
                            #print(idx,"MONTH-YEAR/MONTH-YEAR DATE:",fromDate,"TO",toDate)
                            #else: # If it does not match
                              #  pass # Discard 
                        elif re.match("^([0]?[1-9]|[1][0-2])[./-][/-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$",dateobject):
                            # Month.-Month.Year
                            dateobjectsplit = dateobject.split(sep)
                            dateobjectsplitA = dateobject[0:2]
                            dateobjectsplitB = dateobjectsplit[1]
                            dateobjectsplit = dateobjectsplitB.split(".")
                            Year = dateobjectsplit[1]
                            fromMonth = dateobjectsplitA
                            toMonth = dateobjectsplit[0]
                            fromDate = Year+"-"+fromMonth
                            toDate = Year+"-"+toMonth

                            #print(idx,"Month.-month.year DATE:",fromDate,"TO",toDate)

                        elif re.match("^([0-9]{4}|[0-9]{2})$",dateobject): # Year-only
                            dateobject = dateobject#+"-01-01"
                            if len(dateobject) != 4: # Only allow 4-digit years + 6 digits -01-01.
                                #print(idx,"YEAR ONLY DATE:",dateobject)
                                skipThis = True
                            #else: # If it does not match
                               # skipThis = True
                        elif re.match("^([0-9]{4}|[0-9]{2})[./-]([0-9]{4}|[0-9]{2})$",dateobject): # Year-year
                            dateobject = dateobject.replace("/","-")
                            daterange = dateobject.split("-")
                            fromDate = daterange[0]
                            toDate = daterange[1]
                            if len(fromDate) != 4 and len(toDate) != 4: # Only allow 4-digit years.
                                skipThis = True
                                                        #print(idx,"YEAR-YEAR RANGE:",fromyear,"TO",toyear)
                            #else: # If it does not match

                        else:
                            print(idx,"Dropped",dateobject)
                            datesNotAdded.append(dateobject)
                            skipThis = True
                        if skipThis == False:
                            if fromDate != False and toDate != False:
                                #print(filenamePlain,"dated between",fromDate,"-",toDate)
                                placenamedict[filenamePlain]["fromDate"] = fromDate
                                placenamedict[filenamePlain]["toDate"] = toDate
                            else:
                                #print(filenamePlain,"dated",dateobject)
                                placenamedict[filenamePlain]["date"] = dateobject

            #print(filenamePlain,"|",document)
        i+=1
    #chronology
    print("")
    for dupe in dupes:
        print("Duplicate:",dupe)
    print("Len of j:",len(j),"Len of placenamedict:",len(placenamedict),"\nNo. dropped dates:",len(datesNotAdded))

else:
    print("No chronology file provided. Skipping datetime augmentation.")
    


# Placenames
if os.path.isfile("ID_sted-verdier.xlsx"):
    print("Detected ID_sted-verdier.xlsx")
    if os.path.exists("xml-filer"):
        print("XML>CMIF placename augmentation enabled")
        listXMLfiles = glob.glob("xml-filer/**/*.xml",recursive=True)
        hasXMLs = True
    else:
        print("No XML files provided. XML>CMIF placename augmentation disabled")
else:
    print("No ID_sted-verdier file provided. Skipping placename augmentation.")

Directory 'MXMLM X2' created
Directory 'MXMLM X2\sourcefiles' created
Directory 'MXMLM X2\output' created
Newest chronology file: Kronologi_Munchs_brev_20220923.xlsx
Detected ID_sted-verdier.xlsx
XML>CMIF placename augmentation enabled


# *MunchXMLmuncher* **1.b.2**

## CMIF Metadata & options
Users:
I ask that you do not touch anything below the header **Program** unless you *sort of* know what you're doing. :)

In [2]:
version = "1.b.2" # Describes the "program's" state of completion and versioning.

In [3]:
# Users: only edit things that exist WITHIN double quotation marks ("").
cmifTitle = "eMunch TEI registry and correspondence created with MXMLM "+str(version) # Title of resulting CMIF
editorName = "Loke Sjølie" # The name to issue to the CMIF file as "editor" (responsible for this file)
editorMail = "loke.sjolie@ub.uio.no" # The e-mail associated with the above.

cmifUid = "a403c593-09df-4538-8acf" # Unique ID. Used in sourceDesc of CMIF. Don't change it without a good reason.
# cmifUid is also used as "source" for the time being in each object. Read more about this in CMIF docs.

#publishers = 1 # How many publishers? Add later if required.
publisherURL = "https://www.emunch.no/" # Website of publisher #1
publisherName = "Edvard Munchs Tekster" # Name of publisher #1

cmifURL = "https://www.emunch.no/" # URL where this file is located
typeOfBibl = "online" # The type of bibliography that is being described
publicationStatementFull = "PUBLICATION STATEMENT REQUIRED"


### Init, metadata, etc

In [4]:
today = date.today() # Sett dato i dag
today = today.strftime("%Y-%m-%d") # Formater dato
currVer = version+" "+today

previouslyRun = "Last executed code was version "+str(currVer)+". All OUTPUT files are current to that version on that date.\n"+str(cmifUid)+"."
print("Version",currVer)

Version 1.b.2 2022-10-04


## Program
### Setup

In [5]:
hasCorrespondenceXML,hasTEIXML,hasChrono,hasXMLs = False,False,False,False
programfolder = "MXMLM "+currVer
if os.path.exists(programfolder):
    print("Found directory '% s'" % programfolder)
else:
    os.mkdir(programfolder)
    print("Directory '% s' created" % programfolder)
inputfolder = os.path.join(programfolder,"sourcefiles") # The folder containing the TEI/XML-files to be transformed.
paths = ['correspondence.xml','register_tei.xml']
if not os.path.exists(inputfolder):
    os.mkdir(inputfolder)
    print("Directory '% s' created" % inputfolder)
outputfolder = os.path.join(programfolder,"output") # Output folder
if not os.path.exists(outputfolder):
    os.mkdir(outputfolder)
    print("Directory '% s' created" % outputfolder)
    
lookupChrono = sorted(glob.glob("*Kronologi_Munchs_brev*.xlsx"), key=os.path.getmtime)
x = len(lookupChrono)-1
if x > -1:
    print("Newest chronology file:",lookupChrono[x])
    chronologyFile = lookupChrono[x]
    hasChrono = True
    shutil.copy2(chronologyFile, inputfolder+"/kronologi.xlsx")
    chronologyFile = inputfolder+"/kronologi.xlsx"
else:
    print("No chronology file provided. Skipping datetime augmentation.")
if os.path.isfile("ID_sted-verdier.xlsx"):
    print("Detected ID_sted-verdier.xlsx")
    if os.path.exists("xml-filer"):
        print("XML>CMIF placename augmentation enabled")
        listXMLfiles = glob.glob("xml-filer/**/*.xml",recursive=True)
        hasXMLs = True
        f = open(programfolder+"/xml_directory.txt", "w")
        f.write(str(listXMLfiles))
        f.close()
    else:
        print("No XML files provided. XML>CMIF placename augmentation disabled")
else:
    print("No ID_sted-verdier file provided. Skipping placename augmentation.")

    
for item in paths:
    if os.path.isfile(item):
        a = os.path.getmtime(item)
        if os.path.isfile(inputfolder+"/"+item):
            b = os.path.getmtime(inputfolder+"/"+item)
            print("Found existing instance of",inputfolder+"/"+item)
            if a>b:
                shutil.copy2(item, inputfolder+"/"+item)
                print("\tReplaced older version of",item,"in",inputfolder,"(file's last modified date difference is +"+str(a-b)+str(")"))
            else:
                print("\tUsing existing version of",item,"(file's last modified date difference is",str(a-b)+str(")"))
            if item == "correspondence.xml":
                hasCorrespondenceXML = True
            elif item == "register_tei.xml":
                hasTEIXML = True
        else:
            shutil.copy2(item, inputfolder+"/"+item)
            print("\tYoink! Copied",item,"to",inputfolder)
            if item == "correspondence.xml":
                hasCorrespondenceXML = True
            elif item == "register_tei.xml":
                hasTEIXML = True
    else:
        if os.path.isfile(inputfolder+"/"+item):
            print("\tFound existing file",item,"in the sourcefiles directory, but not in the main directory.")
            if item == "correspondence.xml":
                hasCorrespondenceXML = True
            elif item == "register_tei.xml":
                hasTEIXML = True
        else:
            print("\nWARNING Didn't find",item,"in any of the working directories!\n")
filesForMunching = glob.glob(inputfolder+"/*")
if len(filesForMunching) == 0:
    print("Stop! You need to put some file/s (correspondence.xml,register_tei.xml) in the sourcefiles folder for me to eat!")
    raise KeyboardInterrupt
elif hasTEIXML == False and hasCorrespondenceXML == False:
    print("Critical error! I didn't find either of the correspondence.xml OR register_tei.xml files. Means I don't have anything to munch!")
    raise KeyboardInterrupt
else:
    print("\nSummary:")
    for name in filesForMunching:
        if name == inputfolder+"\\correspondence.xml" or name == inputfolder+"\\register_tei.xml":
            print('\t'+name,'will be used')
        elif name == inputfolder+"\\kronologi.xlsx":
            pass
        else:
            print("\tWARNING Detected unusual file!",name,"may not be a file I can munch!")
    if hasTEIXML == True:
        print("\tPlacename augmentation enabled")
    if hasChrono == True:
        print('\tDate augmentation enabled from',lookupChrono[x])

Directory 'MXMLM 1.b.2 2022-10-04' created
Directory 'MXMLM 1.b.2 2022-10-04\sourcefiles' created
Directory 'MXMLM 1.b.2 2022-10-04\output' created
Newest chronology file: Kronologi_Munchs_brev_20220923.xlsx
Detected ID_sted-verdier.xlsx
XML>CMIF placename augmentation enabled
	Yoink! Copied correspondence.xml to MXMLM 1.b.2 2022-10-04\sourcefiles
	Yoink! Copied register_tei.xml to MXMLM 1.b.2 2022-10-04\sourcefiles

Summary:
	MXMLM 1.b.2 2022-10-04\sourcefiles\correspondence.xml will be used
	MXMLM 1.b.2 2022-10-04\sourcefiles\register_tei.xml will be used
	Placename augmentation enabled
	Date augmentation enabled from Kronologi_Munchs_brev_20220923.xlsx


### Check for XML letter files

In [33]:
if hasXMLs == True: # If the XML files should be used for updating
    placenamedict = collections.defaultdict(dict)
    #placenamedict = {}
    addrsFoundInXMLs = [] # Make a simple list to hold the short names of every file we've found addresses for
    xmlswithnoaddress = []
    plainnames = []
    i=0
    countAddrsD = 0
    countAddrsA = 0
    for item in listXMLfiles:
        find_address = []
        findFileName = item.split("\\") # Make filepath a list
        findFileName = findFileName[len(findFileName)-1] # Get the path destination file
        chkStr = findFileName[0:2]
        if chkStr == "PN":
            filenamePlain = findFileName[0:6] # Must be regulated to remove .xml as well as pagination from filename
            #print(findFileName,filenamePlain)
        elif chkStr == "No":
            filenamePlain = findFileName[0:11] # Must be regulated to remove .xml as well as pagination from filename
            #print(findFileName,filenamePlain)
        else:
            print("PROBLEM")
        if filenamePlain not in plainnames:
            plainnames.append(str(filenamePlain))
        print ("\r","Progress:",round(i/len(listXMLfiles)*100),"%", end='')
        
        if filenamePlain in addrsFoundInXMLs: # if we found the address for this xml
            pass # Skip if we've already found an address for this XML filename
        else:
            with open(item, "r", encoding="utf-8") as file: # Open a file
                letterfile = file.readlines() # Les innholdet som linjer
                letterfile = "".join(letterfile) # Linjene blir kombinert i en variabel
            soup = BeautifulSoup(letterfile, features="xml") # It is now soup
            find_address = soup.findAll("address") # Look for addrline element.
            #augh,addrKey = "","" # Reset
            foundSender,foundRecipient = False,False # Reset

            ## Code below enables seeking address lines in the letters.
            ## It is intentionally disabled at this time because there is no decisive way to get only
            ## the actual addresses of the sender(s) and the recipient(s).
            #if find_address: # If there's an addrline element:
            #    print("Address in",findFileName)
            #    recipNo = 0
            #    senderNo = 0
            #    addrcount = -1
            #    for addr in find_address:
            #        augh,addrKey,isSender = "","","" # Reset
            #        addrcount +=1
            #        if len(find_address) > 1:
            #            print("\n\n",findFileName,"\n\n")
            #        isSender = find_address[addrcount].get("n")
            #        if isSender == "discussed":
            #            pass # Ignore discussed persons/addresses. Reason: irrelevant.
            #        elif not isSender:
            #            pass
            #            #print("IGNORED: Address not tagged as sender or recipient",findFileName)
            #        else:
            #            print(isSender,find_address[addrcount])
            #            try:
            #                augh = find_address[addrcount].findChild("placename", recursive=True) # Get the placename in the addrline
            #                try:
            #                    addrKey = augh.get('key')
            #                    if addrKey:
            #                        countAddrsA+=1
            #                        if isSender == "sender":
            #                            senderNo+=1
            #                            #print("\tSENDER",addrKey)
            #                            placenamedict[filenamePlain]["sender"+str(senderNo)] = addrKey
            #                            foundSender = True
            #                        elif isSender == "recipient":
            #                            recipNo+=1
            #                            placenamedict[filenamePlain]["recipient"+str(recipNo)] = addrKey
            #                            foundRecipient = True
            #                            #print("\tRECIPIENT",addrKey)
            #                        else:
            #                            print("\nWTF?\n",addrKey)
            #                            countAddrsA-=1
            #                        addrsFoundInXMLs.append(filenamePlain) # Add the filename to the list of XMLs already found
            #                    else:
            #                        print("Accessed key, but it was counted as false?",addrKey)
            #                except:
            #                    print("No key in placename",augh)
            #            except:
            #                print("No placename child element")

            ## Code below enables retrieval of an address enclosed in a dateline element.
            ## This is understood to be the sender's address.
            if foundSender == False: # If a recipient has not been found
                find_address = soup.find("dateline") # Look for a dateline element
                if find_address: # If there is a dateline element:
                    #print("Dateline in",findFileName)
                    find_address = find_address.findChild("placename", recursive=True) # Get the placename
                    #print("DATELINE",find_address)
                    try: # There are documents with datelines but no dates in them confirmed.
                        addrKey = find_address.get('key') # Get the internal ID of the placename
                        addrKey = addrKey.replace("pl","") # Remove "pl" prefix
                        addrsFoundInXMLs.append(filenamePlain) # Add the filename to the list of XMLs already found
                        placenamedict[filenamePlain]["sender"] = addrKey
                        foundSender = True
                        #print("\tSENDER",filenamePlain,addrKey)
                        #prefix = "DATELINE" # debug
                        countAddrsD+=1 # Count it

                    except:
                        if filenamePlain not in xmlswithnoaddress:
                            xmlswithnoaddress.append(filenamePlain)
                        #print("Dateline without address in",findFileName)
                else:
                    if filenamePlain not in xmlswithnoaddress:
                        xmlswithnoaddress.append(filenamePlain)
        i+=1
print("")
# Debug
#for xmlname in addrsFoundInXMLs:
#    if xmlname in xmlswithnoaddress:
#        xmlswithnoaddress.remove(xmlname)
#print("# Plain names:",len(plainnames),"\n# With placenames:",len(placenamedict),"# Without:",len(xmlswithnoaddress),"\t",len(xmlswithnoaddress)+len(placenamedict))

 Progress: 100 %rogress: 14 %Progress: 20 %% 25 % %Progress: 26 %Progress: 29 %Progress: 36 %% 38 % 48 % %%Progress: 91 % %


### Read chronology
The chronology's document ID formatting does not match the formatting found everywhere else. This step includes a large portion of data massaging that catches erroneous names ("M N 2714","N 3491"), applies formatting to match the patterns, and checks for duplicates in data.

After correcting bad formatting in names, the script attempts to fetch the dates associated with each where available. The script attempts to correct for the following data formats: DD.MM.YYYY, ??.MM.YYYY, ?D.M?.YYYY, MM.-MM.YYYY, MM-MM.YYYY, YYYY-YYYY and... several more. I have gotten to a point where only the most malformed of dates are allowed to pass by - these include instances of 23.07.1895/1896, 2[6/7].12.1912, and other equally-mangled date formats. After retrieving the date (if capable of doing so), the script reformats all dates to match YYYY-MM-DD format, dropping pairs where incomplete in the order of DD>MM>YYYY to ensure that the resulting data set is as confident as possible.

In [34]:
# This step requires several customised workarounds to handle the irregular and non-machine-readable data present in
# the date file. As a result, it can accept a broad range of dates.
if hasChrono == True:
    j = []
    datesNotAdded = []
    dupes = []
    chronology = pd.read_excel(chronologyFile).dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    chronology = chronology.fillna("N/A")
    try:
        if placenamedict:
            print("Placenamedict found with content")
        else:
            print("Placenamedict found without content")
    except:
        placenamedict = collections.defaultdict(dict)
        print("Made new dict")
    i=0
    for idx,row in chronology.iterrows():
        document = chronology.iloc[idx]['Objektnr.']
        skipThis = False
        if document != "N/A":
            chkStr = document[0:2]
            if chkStr == "PN":
                #print(document)
                document = document.replace(" ","0")
                if len(document) == 5:
                    filenameEdited=document[:2] + "0" + document[len(document)-3:]
                else:
                    filenameEdited = document[:2] +"000"+ document[len(document)-4:]
                    filenameEdited = filenameEdited[:2] + filenameEdited[len(filenameEdited)-4:]
                filenamePlain = filenameEdited[0:6]
            elif chkStr == "MM":
                #No-MM_K0499
                #filenamePlain = document[0:11]
                if len(document) == 8:
                    #print(document)
                    filenamePlain = document[:2]+"_"+document[3]+"0"+document[5:]
                else:
                    filenamePlain = document[:2]+"_"+document[3]+document[5:]
                filenamePlain = "No-"+filenamePlain
            elif chkStr == "M ": # Implemented to catch "M N 2714"...
                filenamePlain = document[:1]+"_"+document[2]+document[4:]
                filenamePlain = "No-M"+filenamePlain
                #filenamePlain = filenamePlain.replace(" ","")
                print("Caught",document,">",filenamePlain)
            elif chkStr == "N ": # Implemented to catch "N 3491"...
                  # This is ASSUMED to be "No-MM_N3491" and not PN3491
                filenamePlain = "No-MM-N_"+document[2:]
                print("Caught",document,">",filenamePlain)
            elif chkStr == " M": # Implemented to catch " MM N 723"...
                filenamePlain = document[1:3]+"_"+document[4]+"0"+document[6:]
                print("Caught",document,">",filenamePlain)
            else:
                print("!!\tError in",document,"length:",len(document))
                filenamePlain = "Formatting error"
            #print(filenamePlain)
            
            if filenamePlain != "Formatting error":
                #date = chronology.iloc[i]['Dato']
                if filenamePlain in j:
                    dupes.append(filenamePlain)
                    #print("Found duplicate",filenamePlain)
                else:
                    j.append(filenamePlain)
                ### This indent for dates after fixing the naming conventions
                fromMonth,fromYear,toMonth,toYear,dateobject,Year,fromDate,toDate = False,False,False,False,False,False,False,False
                #newYear,newMonth,newDay = "","",""
                dateobject = chronology.iloc[idx]['Dato']

                if dateobject != "N/A": # If date object isn't N/A
                    if isinstance(dateobject,datetime.date): # If it's just a datetime object
                        dateobject = dateobject.strftime("%Y-%m-%d") # We're done! Yay!
                        #print(idx,"datetime DATE:",dateobject)
                    else: # If it's not a datetime object:
                        dateobject = str(dateobject)
                        #if ".." in dateobject: 
                        dateobject = dateobject.replace("..",".") # Sometimes there's double dots (..)...
                        dateobject = dateobject.replace("??.","") # Remove trash dates
                        if "?" in dateobject[0:2]:
                            dateobject = dateobject[3:]
                        if "?" in dateobject[0:2]:
                            dateobject = dateobject[3:]
                        #chkDt = dateobject[0:2] # First 2 letters
                        #dateobject = dateobject.replace("/","-")
                        #dateobjectlist = dateobject.split(".")
                        dateobject = dateobject.replace("–","-") # Yes, there are a few dates separated by long dash...
                        if "-" in dateobject: # Used for cases where there may be - or / used as separator.
                            sep = "-"
                        else:
                            sep = "/"
                        if re.match("^([0]?[1-9]|[1|2][0-9]|[3][0|1])[./-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$",dateobject):
                            # Full date
                            dateobjectsplit = dateobject.split(".")
                            dateobject = dateobjectsplit[2]+"-"+dateobjectsplit[1]+"-"+dateobjectsplit[0]
                            if len(dateobjectsplit[2]) != 4: # Only allow 4-digit years.
                                #print(idx,"COMPLETE DATE:",dateobject)
                                skipThis = True
                           # else: # If it does not match
                              #  skipThis = True
                        elif re.match("^([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$",dateobject): # Month.Year
                            dateobjectsplit = dateobject.split(".")
                            dateobject = dateobjectsplit[1]+"-"+dateobjectsplit[0]
                            dateobject = dateobject#+"-01"
                            if len(dateobjectsplit[1]) != 4: # Only allow 4-digit years.
                                #print(idx,"MONTH-YEAR DATE:",dateobject)
                                skipThis = True
                            #else: # If it does not match
                                #skipThis = True 
                        elif re.match("^([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})[./-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$",dateobject): # Month.Year
                            # MM.YYYY/MM.YYYY
                            # MM.YYYY-MM.YYYY
                            dateobjectsplit = dateobject.split(sep)
                            dateobjectsplitFrom = dateobjectsplit[0].split(".")
                            dateobjectsplitTo = dateobjectsplit[1].split(".")
                            fromMonth = dateobjectsplitFrom[0]
                            fromYear = dateobjectsplitFrom[1]
                            toMonth = dateobjectsplitTo[0]
                            toYear = dateobjectsplitTo[1]
                            fromDate = fromYear+"-"+fromMonth
                            toDate = toYear+"-"+toMonth
                            #dateobject = dateobjectsplit[1]+"-"+dateobjectsplit[0]
                            #dateobject = dateobject+"-01"
                            #if len(dateobjectsplit[1]) == 4: # Only allow 4-digit years.
                            #print(idx,"MONTH-YEAR/MONTH-YEAR DATE:",fromDate,"TO",toDate)
                            #else: # If it does not match
                              #  pass # Discard 
                        elif re.match("^([0]?[1-9]|[1][0-2])[./-][/-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$",dateobject):
                            # Month.-Month.Year
                            dateobjectsplit = dateobject.split(sep)
                            dateobjectsplitA = dateobject[0:2]
                            dateobjectsplitB = dateobjectsplit[1]
                            dateobjectsplit = dateobjectsplitB.split(".")
                            Year = dateobjectsplit[1]
                            fromMonth = dateobjectsplitA
                            toMonth = dateobjectsplit[0]
                            fromDate = Year+"-"+fromMonth
                            toDate = Year+"-"+toMonth

                            #print(idx,"Month.-month.year DATE:",fromDate,"TO",toDate)

                        elif re.match("^([0-9]{4}|[0-9]{2})$",dateobject): # Year-only
                            dateobject = dateobject#+"-01-01"
                            if len(dateobject) != 4: # Only allow 4-digit years + 6 digits -01-01.
                                #print(idx,"YEAR ONLY DATE:",dateobject)
                                skipThis = True
                            #else: # If it does not match
                               # skipThis = True
                        elif re.match("^([0-9]{4}|[0-9]{2})[./-]([0-9]{4}|[0-9]{2})$",dateobject): # Year-year
                            dateobject = dateobject.replace("/","-")
                            daterange = dateobject.split("-")
                            fromDate = daterange[0]
                            toDate = daterange[1]
                            if len(fromDate) != 4 and len(toDate) != 4: # Only allow 4-digit years.
                                skipThis = True
                                                        #print(idx,"YEAR-YEAR RANGE:",fromyear,"TO",toyear)
                            #else: # If it does not match
                                
                        else:
                            print(idx,"Dropped",dateobject)
                            datesNotAdded.append(dateobject)
                            skipThis = True
                        if skipThis == False:
                            if fromDate != False and toDate != False:
                                #print(filenamePlain,"dated between",fromDate,"-",toDate)
                                placenamedict[filenamePlain]["fromDate"] = fromDate
                                placenamedict[filenamePlain]["toDate"] = toDate
                            else:
                                #print(filenamePlain,"dated",dateobject)
                                placenamedict[filenamePlain]["date"] = dateobject
                        
            #print(filenamePlain,"|",document)
        i+=1
#chronology
print("")
for dupe in dupes:
    print("Duplicate:",dupe)
print("Len of j:",len(j),"Len of placenamedict:",len(placenamedict),"\nNo. dropped dates:",len(datesNotAdded))


Placenamedict found without content
Caught N 3491 > No-MM-N_3491
902 Dropped 26-27.12.1912
935 Dropped 14.-15.10.1912
1288 Dropped 31.01.-01.02.1909
Caught M N 2714 > No-MM_N2714
2089 Dropped 01.-02.02.1904
2226 Dropped 09.-11.03.1902
Caught  MM N 723 > MM_N0723

Len of j: 2660 Len of placenamedict: 1145 
No. dropped dates: 5


In [35]:
i=0
for item in placenamedict:
    sentFrom = placenamedict[item].get('sender')
    if sentFrom:
        i+=1
print(i)
placenamedict['PN0045']

0


{'fromDate': '1896', 'toDate': '1897'}

In [52]:
testFilePath = ["MXMLM 1.b.2 2022-10-04\sourcefiles\correspondence.xml"]
with open(inputfilepath, "r", encoding="utf-8") as file: # Open a file
    tei = file.readlines() # Les innholdet som linjer
    tei = "".join(tei) # Linjene blir kombinert i en variabel
soup = BeautifulSoup(tei, from_encoding="UTF-8",features="xml") # It is now soup

In [65]:
soup.find(attrs={'xml:id':"No-MM_K2564"})

<div xml:id="No-MM_K2564">
      MM K 2564
      <list type="proveniens">
<item n="ownerMsSig">MM K 2564</item>
<item n="owner">Munchmuseet</item>
</list>
<list type="description">
<item n="sender">
<ref>Emma Krause</ref>
<ref>Verein "Volkswohl"</ref>
</item>
<item><list type="objectType"><item n="letter">brev</item></list></item>
<item>
<list type="date">
<item n="dated">
<date type="year" when="1906">06.04.1906</date>
<date type="month" when="--04"></date>
<date type="day" when="---06"></date>
<seg type="dateCertainty">(sikker datering)</seg>
</item>
</list>
</item>
<item n="language">tysk</item>
</list>
<list type="physDescription">
<item n="paperType">postkort</item>
<item n="medium">håndskrevet med blekk (svart)</item>
</list>
</div>

### Get names of locations
This step transforms the IDs acquired in the XML letter files into proper names. The script attempts to find the most precise name for the location available.

In [48]:
if hasXMLs == True: # This step only affects placenames.
    placenamedf = pd.read_excel("ID_sted-verdier.xlsx").dropna(axis=1, how='all').dropna(axis=0, how='all').reset_index(drop=True)
    placenamedf = placenamedf.fillna("N/A")
    for item in placenamedict:
        print(item)
        try:
            print("\tSent from:",placenamedict[item]['sender'])
            try:
                placenameSearch = placenamedf[placenamedf['ID'].astype(str) == str(placenamedict[item]['sender'])]
                stedsnavn = placenameSearch["sted"].values[0]
                regionnavn = placenameSearch["region, nasjonal"].values[0]
                landnavn = placenameSearch["land"].values[0]
                kontinent = placenameSearch["region, internasjonal"].values[0]
                if stedsnavn != "N/A":
                    print("Sted:",stedsnavn)
                    placenamedict[item]['sender'] = stedsnavn
                else:
                    if regionnavn != "N/A":
                        print("Region:",regionnavn)
                        placenamedict[item]['sender'] = regionnavn
                    else:
                        if landnavn != "N/A":
                            print("Land:",landnavn)
                            placenamedict[item]['sender'] = landnavn
                        else:
                            if kontinent != "N/A":
                                print("Kontinent:",kontinent)
                                placenamedict[item]['sender'] = kontinent
                            else:
                                print("Fant ikke stedsnavn.")
            except:
                print("No match for",str(placenamedict[item]['sender']))
        except:
            pass # Generally means that the entry has a date but not a location, and thus there is no need.
        try: # Try finding the date value
            date = placenamedict[item]['date']
            print("\tDated:",date)
        except: # No date value found
            try: # Is there a fromdate and todate?
                date = str(placenamedict[item]['fromDate'])+" to "+str(placenamedict[item]['toDate'])
                print("\tDated between:",date)
            except: # There is no date value and no fromdate and todate
                pass # Skip it
    #print("\n")

No-MM_N01148
	Dated: 1932-11
No-MM_N00275
	Dated: 1932-11
No-MM_N01697
	Dated: 1932-10
No-MM_N02539
	Dated: 1932-08
PN1428
	Dated: 1932-06
PN1421
	Dated: 1932-06
No-MM_N01149
	Dated: 1932-04
PN1429
	Dated: 1932-02
No-MM_N03376
	Dated: 1932-02
PN0690
	Dated: 1932-01
No-MM_N03121
	Dated: 1932-01
No-MM_N01728
	Dated between: 1931 to 1933
No-MM_N01729
	Dated between: 1931 to 1933
No-MM_N01668
	Dated: 1931
PN1437
	Dated: 1931-12
No-MM_N02778
	Dated: 1931-12
No-MM_N02538
	Dated: 1931-12
No-MM_N02022
	Dated between: 1931-10 to 1931-11
PN0688
	Dated: 1931-10
PN1096
	Dated: 1931-10
PN1094
	Dated: 1931-06
No-MM_N03752
	Dated: 1931-06
No-MM_N01145
	Dated: 1931-05
No-MM_N01143
	Dated: 1931-05
PN0097
	Dated: 1931-02
PN0092
	Dated between: 1931-01 to 1931-02
No-MM_N02535
	Dated between: 1931-01 to 1931-02
PN0094
	Dated: 1931-01
PN0093
	Dated: 1931-01
No-MM_N02350
	Dated: 1930-09
No-MM_N03648
	Dated: 1930-08
No-MM_N02313
	Dated: 1930-08
No-MM_N02310
	Dated: 1930-08
No-MM_N01147
	Dated: 1930-06
No-MM_

	Dated: 1908-10
No-MM_N0939
	Dated: 1908-10
PN1183
	Dated: 1908-10
PN0486
	Dated: 1908-10
No-MM_N1931
	Dated: 1908-10
PN0485
	Dated: 1908-09
No-MM_N1941
	Dated: 1908-09
No-MM_N1951
	Dated: 1908-09
No-MM_N1969
	Dated: 1908-09
No-MM_N1947
	Dated: 1908-09
PN0483
	Dated between: 1908-08 to 1908-09
PN0481
	Dated: 1908-08
No-MM_N2770
	Dated: 1908-06
No-MM_N1715
	Dated: 1908-06
No-MM_N1760
	Dated: 1908
No-MM_N2031
	Dated: 1908-06
PN0477
	Dated: 1908-06
No-MM_N2029
	Dated: 1908-05
PN0476
	Dated: 1908-05
No-MM_N3023
	Dated: 1908-05
No-MM_N3671
	Dated: 1908-05
No-MM_N3030
	Dated: 1908-05
PN1332
	Dated: 1908
No-MM_N1904
	Dated: 1908-05
PN0091
	Dated: 1908-05
No-MM_N3344
	Dated: 1908
No-MM_N2194
	Dated: 1908-04
No-MM_N2326
	Dated: 1908-04
No-MM_N2057
	Dated: 1908
No-MM_N2195
	Dated between: 1908-03 to 1908-04
No-MM_N2196
	Dated: 1908
No-MM_N3053
	Dated: 1908-03
No-MM_N2254
	Dated: 1908-03
No-MM_N0936
	Dated: 1908-03
No-MM_N0266
	Dated between: 1907 to 1908
No-MM_N1911
	Dated between: 1907 to 1908


No-MM_N1858
	Dated: 1899
No-MM_N1829
	Dated: 1899
No-MM_N1827
	Dated: 1899
No-MM_N1828
	Dated: 1899
No-MM_N1826
	Dated: 1899
No-MM_N1825
	Dated: 1899-10
No-MM_N1824
	Dated: 1899-09
No-MM_N1854
	Dated: 1899
No-MM_N1823
	Dated: 1899-08
No-MM_N1821
	Dated: 1899
No-MM_N1820
	Dated: 1899
No-MM_N1819
	Dated: 1899
No-MM_N1818
	Dated: 1899
PN1005
	Dated: 1899-06-24
No-MM_N1817
	Dated: 1899
No-MM_N1816
	Dated: 1899-06
No-MM_N1815
	Dated: 1899-05
No-MM_N1814
	Dated: 1899-05-25
No-MM_N1813
	Dated: 1899-05-25
No-MM_N1812
	Dated: 1899
No-MM_N0830
	Dated: 1899-05-08
No-MM_N1811
	Dated between: 1899-04 to 1899-05
No-MM_N0829
	Dated: 1899-04-25
No-MM_N1810
	Dated: 1899
No-MM_N1809
	Dated: 1899-04-22
No-MM_K4548
	Dated: 1899-04-21
No-MM_N0831
	Dated: 1899-04
No-MM_N1808
	Dated: 1899-04
No-MM_N1807
	Dated: 1899-04
No-MM_N1806
	Dated: 1899
No-MM_N1805
	Dated between: 1898 to 1899
PN0275
	Dated: 1898-07
No-MM_N3416
	Dated: 1898-07
PN1040
	Dated: 1898-06-13
PN0023
	Dated: 1898-03
PN1448
	Dated: 1898-03-22


### Read/process TEI-XML

In [50]:
# Create CMIF boilerplate object
CMIFstring = '<?xml-model href="https://raw.githubusercontent.com/TEI-Correspondence-SIG/CMIF/master/schema/cmi-customization.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?><TEI xmlns="http://www.tei-c.org/ns/1.0"><teiHeader><fileDesc><titleStmt><title>'+str(cmifTitle)+'</title><editor>'+str(editorName)+'<email>'+str(editorMail)+'</email></editor></titleStmt><publicationStmt><publisher><ref target="'+str(publisherURL)+'">'+str(publisherName)+'</ref></publisher><idno type="url">'+str(cmifURL)+'</idno> <date when="'+str(today)+'"/><availability><licence target="https://creativecommons.org/licenses/by/4.0/">This file is licensed under the terms of the Creative-Commons-License CC-BY 4.0</licence></availability></publicationStmt><sourceDesc><bibl type="'+str(typeOfBibl)+'" xml:id="'+str(cmifUid)+'">'+str(publicationStatementFull)+'</bibl></sourceDesc></fileDesc><profileDesc><dummy/></profileDesc></teiheader><body><p/></body></text></tei>'
CMIF = BeautifulSoup(CMIFstring,"xml") # Read as XML, not HTML

In [53]:
start = time.time()

# DEBUGGER LISTS
documentsWithBadDates = []
documentsWithNoAuthor = []
documentsWithNoRecipient = []
documentsLackingTargetReference = []

# Declare variables
# ---------------------------------------------------------------
noOfLettersToMunch = 0
changedDatesList,addedPlaceList = [],[]
errors_found = [] # List of errors found during execution
letterCount = 0 # # letters, as defined by an item having a recipient, processed
miscCount = 0 # non-letter documents, as defined by an item having no recipients, processed
addresseesUnique = [] # List of unique recipients
datetype = 0 # Var for the type of date we're dealing with
noOfRecipients = 0 # Counting non-unique recipients
otherMiscDocCount = 0 # Counting objects that are not letters.
authorID = "" # Reserved for VIAF etc.
# ---------------------------------------------------------------
iii = 0
#for path in glob.glob(inputfolder+"/*.xml"):
for path in testFilePath:
    inputfilepath = path
    with open(inputfilepath, "r", encoding="utf-8") as file: # Open a file
        tei = file.readlines() # Les innholdet som linjer
        tei = "".join(tei) # Linjene blir kombinert i en variabel
    soup = BeautifulSoup(tei, from_encoding="UTF-8", features="xml") # It is now soup
    # Don't worry about the error screaming about Unicode markup being provided twice
    print("Souped",inputfilepath)
    # Before handling the data: remove all comments
    # Making a list of <!--comments--> to be destroyed...
    commentDocs = 0 # Used only in terminating comments
    comments = 0 # Used only in terminating comments
    for comment in soup.findAll(string=lambda text: isinstance(text, Comment)):
        if "xml:id=\"" in comment:
            commentDocs+=1
        comment.extract()
        comments+=1
    if comments > 0:
        print("Destroyed",comments,"<!--comments-->, of which",commentDocs,"contained an @XML:ID.")
    # ... and checking it twice.
    comments = soup.findAll(string=lambda text: isinstance(text, Comment))
    if comments:
        print("There are still",len(comments),"comments present.")
    else:
        print("All comments destroyed.")
    # Limit workspace to individual div (document) here.
    profileDescElement = CMIF.find('profileDesc') # Target correspondence wrapper
    # For each Div element with an XML:ID (should be each document)
    for document in soup.findAll("div", {"xml:id":True}):
        #print ("\r","Progress:",round(i/len(listXMLfiles)*100),"%", end='')
        # Get the document ID from the <div> element.
        
        authorID,authorName,recipient,recipientID = False,False,False,False # debug
        
        # Look for the document type assignment.
        documentType = document.find("list", {"type" : "objectType"}).findChild(True, recursive=True)#.attrs['n']
        if "brev" in documentType or "letter" in documentType: # Checks if the words "letter" or "brev" appear in the type
            # This code applies to letters as directed by the data type.
            documentID = list(document.attrs.values())[0]
            #print(documentType)
            #print("DEBUG Checking",documentID)
            # Check if the document has more than 0 recipients. If there are no recipients, there is no correspAction required.
            

            # Check if the document has an author.
            if "correspondence" in path:
                authorNameList = document.find("item", {"n":"sender"}).findChildren(True, recursive=True)
                ji=0
                for name in authorNameList:
                    authorName = authorNameList[ji]
                    #authorName = document.find("item", {"n":"sender"})
                    try:
                        targetRef = authorName['target']
                    except:
                        targetRef = "NONE"
                        if documentID not in documentsLackingTargetReference:
                            documentsLackingTargetReference.append(documentID)
                    ji+=1
                recipient = "Edvard Munch"
                
            else:
                authorName = document.find("item", {"n":"author"})
                recipient = document.find("item", {"n":"recipient"})
            if authorName:
                #print(authorName)
                try:
                    authorName = authorName.contents[0]
                except:
                    authorName = "No author"
                    print("WARNING:",documentID,"suffered code 201881X - no author!")
                    errors_found.append("INFO 201881 in "+str(documentID))
                    documentsWithNoAuthor.append(documentID)
            else:
                authorName = "No author"
                print("WARNING:",documentID,"suffered code 201881 no author found!")
                errors_found.append("INFO 201881 in "+str(documentID))
                documentsWithNoAuthor.append(documentID)
            if authorName == "Edvard Munch":
                authorID = "https://viaf.org/viaf/61624802/"
            else:
                authorID = targetRef

            # Attempt to divine the date or date range of the document. Assumes that each document only has 1 date (or 1 range).
            
            ## "BUG" in version Pre-release 1.0:
            ## Dates in correspondence.xml cause error cascades here.
            isDocumentUndated = document.find("item", {"n":"undated"})
            if isDocumentUndated:
                date = "s.d."
                datetype = "none"
            else:
                isDocumentFromTo = document.find("date", {"from":True}) # Does the date element have a from assignment? 
                # Using "from" because PN1350 does not have a fromTo attr despite using fromTo. Uses "from", though. Works fine.
                if isDocumentFromTo: # If it does, and thus has a range (JK, No-MM_T1296 has FROM attr but not a TO attr.)
                    doesDocumentHaveToDate = document.find("date", {"to":True})
                    if doesDocumentHaveToDate:
                        fromDate = isDocumentFromTo['from'] # Extract 'from' date. 
                        #date = " ".join(date)
                        toDate = isDocumentFromTo['to'] # Extract 'to' date.
                        datetype = "range"
                    else:
                        date = isDocumentFromTo['from']
                        fromDate = isDocumentFromTo['from']
                        datetype = "fromRange"
                        if documentID not in documentsWithBadDates:
                            documentsWithBadDates.append(documentID)

                else: # If it doesn't:
                    yearSent = document.find("date", {"type":"year","when":True}) # Check for year element
                    monthSent = document.find("date", {"type":"month","when":True}) # Check for month element
                    daySent = document.find("date", {"type":"day","when":True}) # Check for day element
                    if yearSent:
                        datetype = "exact"
                        date = yearSent.attrs["when"]
                        if monthSent: # Only look for a month if there's a year. That 1 letter with just month/day, tho...
                            M = re.sub('[-]', '', monthSent.attrs["when"]) # Strip the random '-' characters in here.
                            date+="-"+str(M) # Join month to year by YYYY-MM.
                            if daySent: # Only applies if there is a month AND a day. No point having a day if you don't have a month.
                                M = re.sub('[-]', '', daySent.attrs["when"]) # Strip the random '-' characters in here, too.
                                date+="-"+str(M) # Join month to year-month by YYYY-MM-DD.
                    else: # If it doesn't have a year, make one last check
                        doesDocumentHaveToDate = document.find("date", {"to":True}) # if the date just has a to date...
                        if documentID not in documentsWithBadDates:
                                documentsWithBadDates.append(documentID)
                        if doesDocumentHaveToDate:
                            datetype = "toRange"
                        else:
                            print("WARNING:",documentID,"suffered code 301881 - no year found in a specific-year element. Expected in MM_N1071 and MM_N3734.")
                            errors_found.append("INFO 301881 in "+str(documentID))
                            datetype = "none"
                            date = "s.d."
                            #print("WARNING:",documentID,"suffered code 301881 - no year found in a specific-year element. Expected in MM_N1071 and MM_N3734.")
                            #errors_found.append("INFO 301881 in "+str(documentID))

            # Construct CMIF author ("sent") element
            correspDescElement = soup.new_tag("correspDesc", attrs={"key":str(documentID), "ref":"https://www.emunch.no/HYBRID"+str(documentID)+".xhtml", "source":cmifUid})
            profileDescElement.append(correspDescElement)
            if "correspondence" in path:
                for name in authorNameList:
                    targetElementCorrespDesc = CMIF.find("correspDesc", attrs={"key":str(documentID)})
                    correspActionElement = soup.new_tag("correspAction", attrs={'type':'sent'})
                    targetElementCorrespDesc.append(correspActionElement)
                    correspActionTarget = targetElementCorrespDesc.findChild("correspAction",attrs={"type": "sent"}, recursive=False)
                    if authorID != "NONE":
                        persNameElement = soup.new_tag("persName", attrs={"ref":authorID})
                    else:
                        persNameElement = soup.new_tag("persName")
                    persNameElement.string = str(authorName)

                    correspActionTarget.append(persNameElement)
            else:
                targetElementCorrespDesc = CMIF.find("correspDesc", attrs={"key":str(documentID)})
                correspActionElement = soup.new_tag("correspAction", attrs={'type':'sent'})
                targetElementCorrespDesc.append(correspActionElement)
                correspActionTarget = targetElementCorrespDesc.findChild("correspAction",attrs={"type": "sent"}, recursive=False)
                if authorID != "NONE":
                    persNameElement = soup.new_tag("persName", attrs={"ref":authorID})
                else:
                    persNameElement = soup.new_tag("persName")
                persNameElement.string = str(authorName)

                correspActionTarget.append(persNameElement)
                
            if documentID in placenamedict:
                senderLoc = placenamedict[documentID].get('sender')
                if senderLoc:
                    senderLocEle = soup.new_tag("placeName", attrs={"ref":"PLACEIDREF"+str(senderLoc)}) # Create place element
                    senderLocEle.string = str(senderLoc) # Give it a string value (placename)
                    correspActionTarget.append(senderLocEle) # Append the new element to the correspAction element
                    addedPlaceList.append(documentID)
                senderDate = placenamedict[documentID].get('date')
                if senderDate:
                    # Overwrite existing data, if any
                    datetype = "exact"
                    date = senderDate
                    changedDatesList.append(documentID)
                else:
                    senderFromDate = placenamedict[documentID].get('fromDate')
                    senderToDate = placenamedict[documentID].get('toDate')
                    if senderFromDate and senderToDate:
                        datetype = "range"
                        fromDate = senderFromDate
                        toDate = senderToDate
                        changedDatesList.append(documentID)
                        #print(senderFromDate,senderToDate) # Modify the date (range) here
                    else:
                        pass
            
            if datetype == "exact":
                dateSentElement = soup.new_tag("date", attrs={"when":date})
                #print(datetype,date)
            elif datetype == "range":
                dateSentElement = soup.new_tag("date", attrs={"from":fromDate,"to":toDate})
                #print(datetype,fromDate,toDate)
            elif datetype == "fromRange":
                dateSentElement = soup.new_tag("date", attrs={"from":fromDate})
            elif datetype == "toRange":
                dateSentElement = soup.new_tag("date", attrs={"to":fromDate})
                #print(datetype,fromDate)
            elif datetype == "none":
                #print("> NO DATE!",documentID)
                pass
            else:
                print("ERROR 2839 - Unrecognized datetype!")
                errors_found.append("2839")
            if datetype == "none":
                pass
            else:
                # Append date element to correspAction @sent
                correspActionTarget.append(dateSentElement)



            if recipient: # If there are more than 0 recipients:
                letterCount += 1
                i=0
                if recipient == "Edvard Munch":
                    recipientID = "https://viaf.org/viaf/61624802/"
                    recipientType = "persName"
                    recipientName = recipient
                    noOfRecipients += 1
                    noOfLettersToMunch += 1
                    if recipientName not in addresseesUnique:
                        addresseesUnique.append(recipientName)
                    correspActionElement = soup.new_tag("correspAction", attrs={'type':'received'})
                    targetElementCorrespDesc.append(correspActionElement)
                    correspActionTarget = targetElementCorrespDesc.findChildren("correspAction",attrs={"type": "received"}, recursive=False)

                    persNameElement = soup.new_tag("persName", attrs={"ref":recipientID})

                    persNameElement.string = str(recipientName)
                    correspActionTarget[i].append(persNameElement)
                    
                else:
                    recipientList = recipient.findChildren(True) # Get ALL children of the recipient item element. Might be 2+!
                    for each in recipientList: # For every recipient:
                        recipientName = str(each.contents[0]) # Assign a name
                        noOfRecipients += 1
                        if recipientName not in addresseesUnique:
                            addresseesUnique.append(recipientName)
                        recipientID = recipientList[i].attrs["target"] # Assign an ID

                        if "institution" in recipientID:
                            recipientType = "orgName"
                        elif "person" in recipientID:
                            recipientType = "persName"
                        else:
                            print("WARNING:",documentID,"suffered error 20191. Defaulted to person.")
                            recipientType = "persName"
                            errors_found.append("WARNING 20191 in "+str(documentID))

                        correspActionElement = soup.new_tag("correspAction", attrs={'type':'received'})
                        targetElementCorrespDesc.append(correspActionElement)
                        correspActionTarget = targetElementCorrespDesc.findChildren("correspAction",attrs={"type": "received"}, recursive=False)

                        if recipientType == "persName":
                            persNameElement = soup.new_tag("persName", attrs={"ref":recipientID})
                        elif recipientType == "orgName":
                            persNameElement = soup.new_tag("orgName", attrs={"ref":recipientID})

                        persNameElement.string = str(recipientName)
                        correspActionTarget[i].append(persNameElement)
                        i+=1
            else: # If document does not have a recipient, what do we do?
                miscCount+=1
                if documentID not in documentsWithNoRecipient:
                    documentsWithNoRecipient.append(documentID)
        else:
            otherMiscDocCount += 1
        #iii+=1
        #if iii > 30:
         #   raise KeyboardInterrupt
            #print("Skipped item",documentID,"as it is not a letter.")
    #print("</profileDesc>")
end = time.time()

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:226: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Souped MXMLM 1.b.2 2022-10-04\sourcefiles\correspondence.xml
Destroyed 15 <!--comments-->, of which 7 contained an @XML:ID.
All comments destroyed.


In [54]:
for document in documentsWithNoAuthor:
    erasing = CMIF.find("correspDesc",attrs={"key":document})
    print("Decomposing",erasing)
    erasing.decompose() # Obliterate items with no author - this is an err
erasing = CMIF.find("dummy")
erasing.decompose()

Decomposing <correspDesc key="No-MM_K3421" ref="https://www.emunch.no/HYBRIDNo-MM_K3421.xhtml" source="a403c593-09df-4538-8acf"><correspAction type="sent"><persName>No author</persName><date when="1922-07-04"></date></correspAction><correspAction type="received"><persName ref="https://viaf.org/viaf/61624802/">Edvard Munch</persName></correspAction></correspDesc>


In [60]:
print(CMIF)

<?xml version="1.0" encoding="utf-8"?>
<?xml-model href="https://raw.githubusercontent.com/TEI-Correspondence-SIG/CMIF/master/schema/cmi-customization.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?><TEI xmlns="http://www.tei-c.org/ns/1.0"><teiHeader><fileDesc><titleStmt><title>eMunch TEI registry and correspondence created with MXMLM 1.b.2</title><editor>Loke Sjølie<email>loke.sjolie@ub.uio.no</email></editor></titleStmt><publicationStmt><publisher><ref target="https://www.emunch.no/">Edvard Munchs Tekster</ref></publisher><idno type="url">https://www.emunch.no/</idno> <date when="2022-10-04"/><availability><licence target="https://creativecommons.org/licenses/by/4.0/">This file is licensed under the terms of the Creative-Commons-License CC-BY 4.0</licence></availability></publicationStmt><sourceDesc><bibl type="online" xml:id="a403c593-09df-4538-8acf">PUBLICATION STATEMENT REQUIRED</bibl></sourceDesc></fileDesc><profileDesc><correspDesc key="No-MM_K000

In [ ]:
print("Processed",otherMiscDocCount+letterCount,"documents.",str(letterCount)+"("+str(round(letterCount/(otherMiscDocCount+letterCount)*100))+"%) were letters addressed to "+str(noOfRecipients)+" recipients, of which "+str(len(addresseesUnique))+" were unique (meaning each person received avg. "+str(round(letterCount/len(addresseesUnique)))+" letters), and",miscCount,"letters without recipients (if this > 0, there's a problem) in",round(end - start,1),"seconds.")
#print("The register file had 5443 documents of which 2711 were letters. Munch received",noOfLettersToMunch,"letters according to my script.")
if len(errors_found) > 0:
    i = 0
    print("\n"+str(len(errors_found)),"data warnings and errors, listed as INFO, WARNING, and ERROR in order of severity:")
    for error in errors_found:
        i+=1
        if "201881" in error:
            print(i,error,"\n\tDocument has no author. Registered as \"No author\".")
        elif "301881" in error:
            print(i,error,"\n\tDocument has a specific date type, but does not specify or suggest a year (MM-DD/MM). Document has been given \"undated\" status.")
        elif "301882" in error:
            print(i,error,"\n\tCatastrophic failure in date format or harvesting. The script was not designed for this.")
        elif "30190" in error:
            print(i,error,"\n\tCatastrophic failure in recipient list processing. I don't think the script will run to this point with such an error.")
        elif "20191" in error:
            print(i,error,"\n\tThe recipient is not a person or an organization. Suggests error in reference XMLURI. Defaulted to person.")
        else:
            print("There is an error that is not indexed. :(")
        print("\n")
else:
    print("No warnings or errors found.")
print("Saving to disk.")
start = time.time()
with open(outputfolder+"\CMIF_Unified.xml", "w", encoding="utf-8") as output_file:
    output_file.write(CMIF.prettify())
end = time.time()
print("Prettified CMIF file created in",round(end - start,1),"seconds.")
print("Process complete.")